In [1]:
import pandas as pd

In [2]:
adp = pd.read_csv("./datasets/ADP.csv")
fpts = pd.read_csv("./datasets/FPTS_PROJ.csv")

In [3]:
replacement_players = {"RB": "",
                       "QB": "",
                       "WR": "",
                       "TE": ""
            }

for _, row in adp[:100].iterrows():
    position = row["POS"]
    player = row["PLAYER"]
    replacement_players[position] = player
    
replacement_players

{'RB': 'Matt Breida',
 'QB': 'Carson Wentz',
 'WR': 'Sterling Shepard',
 'TE': 'Jared Cook'}

In [4]:
replacement_values = {"RB": 0,
                      "QB": 0,
                      "WR": 0,
                      "TE": 0,
                      "K": 100
            }

for position, player in replacement_players.items():
    if position in ["RB", "QB", "TE", "WR"]:
        replacement_values[position] = fpts.loc[fpts["PLAYER"]==player].values[0,-1]

replacement_values

{'RB': 139.0, 'QB': 274.5, 'WR': 183.8, 'TE': 146.2, 'K': 100}

In [5]:
fpts["VOR"] = fpts.apply(
    lambda row: row["FPTS"] - replacement_values[row['POS']], axis=1
)
fpts.head()

,PLAYER,POS,FPTS,VOR
0,Christian McCaffrey,RB,375.0,236.0
1,Lamar Jackson,QB,361.1,86.6
2,Patrick Mahomes,QB,345.0,70.5
3,Michael Thomas,WR,333.1,149.3
4,Dak Prescott,QB,318.4,43.9


In [6]:
df = fpts.sort_values("VOR", ascending=False)
df["VALUE_RANK"] = df["VOR"].rank(ascending=False)
df.head(50)

,PLAYER,POS,FPTS,VOR,VALUE_RANK
0,Christian McCaffrey,RB,375.0,236.0,1.0
6,Saquon Barkley,RB,310.7,171.7,2.0
8,Ezekiel Elliott,RB,299.6,160.6,3.0
11,Alvin Kamara,RB,296.2,157.2,4.0
13,Dalvin Cook,RB,289.9,150.9,5.0
3,Michael Thomas,WR,333.1,149.3,6.0
21,Derrick Henry,RB,261.4,122.4,7.0
27,Clyde Edwards-Helaire,RB,254.1,115.1,8.0
9,Davante Adams,WR,298.8,115.0,9.0
30,Austin Ekeler,RB,252.6,113.6,10.0


In [7]:
df.to_csv("./datasets/VOR.csv")

In [8]:
adp.head()

,PLAYER,POS,AVG,ESPN
0,Christian McCaffrey,RB,1.0,1.0
1,Saquon Barkley,RB,2.0,2.0
2,Ezekiel Elliott,RB,3.0,3.0
3,Dalvin Cook,RB,4.7,4.0
4,Michael Thomas,WR,5.0,6.0


In [9]:
adp["ADP_RANK"] = adp["AVG"].rank(method="first") # So we don't have ties

df = df.merge(adp, on=["PLAYER", "POS"], how="left")
df.head()

,PLAYER,POS,FPTS,VOR,VALUE_RANK,AVG,ESPN,ADP_RANK
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,1.0
1,Saquon Barkley,RB,310.7,171.7,2.0,2.0,2.0,2.0
2,Ezekiel Elliott,RB,299.6,160.6,3.0,3.0,3.0,3.0
3,Alvin Kamara,RB,296.2,157.2,4.0,5.3,5.0,6.0
4,Dalvin Cook,RB,289.9,150.9,5.0,4.7,4.0,4.0


In [10]:
with pd.option_context('display.max_rows', None):
    display(df.head(100))

,PLAYER,POS,FPTS,VOR,VALUE_RANK,AVG,ESPN,ADP_RANK
0,Christian McCaffrey,RB,375.0,236.0,1.0,1.0,1.0,1.0
1,Saquon Barkley,RB,310.7,171.7,2.0,2.0,2.0,2.0
2,Ezekiel Elliott,RB,299.6,160.6,3.0,3.0,3.0,3.0
3,Alvin Kamara,RB,296.2,157.2,4.0,5.3,5.0,6.0
4,Dalvin Cook,RB,289.9,150.9,5.0,4.7,4.0,4.0
5,Michael Thomas,WR,333.1,149.3,6.0,5.0,6.0,5.0
6,Derrick Henry,RB,261.4,122.4,7.0,7.3,7.0,7.0
7,Clyde Edwards-Helaire,RB,254.1,115.1,8.0,16.0,8.0,17.0
8,Davante Adams,WR,298.8,115.0,9.0,11.3,15.0,8.0
9,Austin Ekeler,RB,252.6,113.6,10.0,15.0,21.0,14.0


In [11]:
df["SLEEPER_SCORE"] = df["VALUE_RANK"] - df["ADP_RANK"]

# Only including players in draft pool
# 12 teams * 16 draft spots = 192 draft picks
overvalued = df.loc[df["AVG"]<192].sort_values(by="SLEEPER_SCORE", ascending=False)
overvalued.head(25)

,PLAYER,POS,FPTS,VOR,VALUE_RANK,AVG,ESPN,ADP_RANK,SLEEPER_SCORE
330,Ke'Shawn Vaughn,RB,17.6,-121.4,331.0,144.3,183.0,141.0,190.0
280,Denzel Mims,WR,77.2,-106.6,281.0,176.5,NaN,176.0,105.0
211,Alshon Jeffery,WR,106.3,-77.5,212.0,149.0,NaN,149.0,63.0
191,Mecole Hardman,WR,123.0,-60.8,192.0,136.0,128.0,134.0,58.0
239,LeSean McCoy,RB,48.8,-90.2,240.0,188.0,208.0,187.0,53.0
234,John Ross,WR,96.6,-87.2,235.0,182.0,NaN,185.0,50.0
221,Parris Campbell,WR,102.8,-81.0,222.0,177.3,174.0,180.0,42.0
162,Alexander Mattison,RB,94.5,-44.5,163.5,126.3,130.0,122.0,41.5
155,Justin Jefferson,WR,143.9,-39.9,156.0,120.7,136.0,118.0,38.0
120,Deebo Samuel,WR,164.9,-18.9,121.0,86.3,86.0,83.0,38.0


In [12]:
undervalued = df.loc[df["AVG"]<192].sort_values(by="SLEEPER_SCORE", ascending=True)
undervalued.head(25)

,PLAYER,POS,FPTS,VOR,VALUE_RANK,AVG,ESPN,ADP_RANK,SLEEPER_SCORE
41,Antonio Gibson,RB,182.9,43.9,42.0,136.0,137.0,133.0,-91.0
101,Cam Newton,QB,269.3,-5.2,102.0,177.0,117.0,177.0,-75.0
127,Ian Thomas,TE,124.6,-21.6,128.0,189.0,NaN,190.0,-62.0
104,Chris Herndon IV,TE,136.6,-9.6,105.0,161.0,156.0,161.0,-56.0
130,Gardner Minshew II,QB,251.5,-23.0,131.0,177.0,185.0,178.0,-47.0
114,Jonnu Smith,TE,130.1,-16.1,115.0,159.0,189.0,159.0,-44.0
146,Teddy Bridgewater,QB,239.2,-35.3,147.0,188.7,193.0,189.0,-42.0
128,Blake Jarwin,TE,123.8,-22.4,129.0,167.3,190.0,166.0,-37.0
110,Dallas Goedert,TE,132.3,-13.9,110.5,148.3,184.0,147.0,-36.5
49,Tarik Cohen,RB,174.8,35.8,50.0,87.3,87.0,85.0,-35.0


In [13]:
overvalued.to_csv("./datasets/OVERVALUED.csv")
undervalued.to_csv("./datasets/UNDERVALUED.csv")
df.to_csv("./datasets/VOR.csv")